In [ ]:
from selenium import webdriver
import pandas as pd
import time
import os
import chromedriver_autoinstaller
import numpy as np
from bs4 import BeautifulSoup
import os
import re

In [ ]:
def open_chromedriver():
    chrome = chromedriver_autoinstaller.install(os.getcwd())

    options = webdriver.ChromeOptions()
    # options.headless = True

    driver = webdriver.Chrome(options=options)
    return(driver)

In [ ]:
def get_seller_name():
    name=[]
    for i in driver.find_elements_by_xpath('//*[@id="section-price"]/ul/li[*]/div[1]/div/div[1]/span[1]/span[1]'):
        name.append(i.text)
    return(name)

In [ ]:
def get_price():
    price=[]
    for i in driver.find_elements_by_xpath('//*[@id="section-price"]/ul/li[*]/div[1]/div/div[1]/span[2]/span'):
        price.append(re.sub('원|,','',i.text))
    return(price)

In [ ]:
def get_delivery_price():
    delivery_price=[]
    for i in driver.find_elements_by_xpath('//*[@id="section-price"]/ul/li[*]/div[1]/div/div[2]/div[2]/span'):
        delivery_price.append(re.sub('배송비\n|,','',i.text))
    return(delivery_price)

In [ ]:
driver = open_chromedriver()

In [ ]:
driver.get('https://msearch.shopping.naver.com/catalog/25166275522')

In [ ]:
seller_name=[]
price=[]
delivery_price=[]
option=[]
title=[]

In [ ]:
num=0
if len(driver.find_elements_by_xpath('//*[@id="section-price"]/div[4]/div/div/button[*]'))==0:
    
    try:
        option_text=driver.find_element_by_xpath('//*[@id="section-price"]/div[4]/em').text
    except:
        option_text= '옵션 없음'
    seller_name.extend(get_seller_name())
    price.extend(get_price())
    delivery_price.extend(get_delivery_price())
    option.extend([option_text]*len(get_price()))
    
    title_base=driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[1]/h2').text
    title = [title_base]*len(price)
    
else:
    for i in driver.find_elements_by_xpath('//*[@id="section-price"]/div[4]/div/div/button[*]'):
        i.click()
        time.sleep(0.5)

        option_text=re.findall('[0-9]+개',i.text)

        seller_name.extend(get_seller_name())
        price.extend(get_price())
        delivery_price.extend(get_delivery_price())
        option.extend(option_text*len(get_price()))

        num=num+1
        print(num,'/',len(driver.find_elements_by_xpath('//*[@id="section-price"]/div[4]/div/div/button[*]')))
    title_base=driver.find_element_by_xpath('//*[@id="__next"]/div/div[2]/div/div[1]/h2').text
    title = [title_base]*len(price)



In [ ]:
pd.DataFrame({'상품명':title,'옵션':option,'판매처':seller_name,'판매가':price,'배송비':delivery_price})